#  Imports

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

# Load & Merge Data


In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
features = pd.read_csv("features.csv")
stores = pd.read_csv("stores.csv")

def preprocess(df, is_train=True):
    df = pd.merge(df, features, on=["Store", "Date", "IsHoliday"], how="left")
    df = pd.merge(df, stores, on="Store", how="left")
    df["Date"] = pd.to_datetime(df["Date"])
    df["Year"] = df["Date"].dt.year
    df["Month"] = df["Date"].dt.month
    df["Week"] = df["Date"].dt.isocalendar().week.astype(int)
    df["Type"] = df["Type"].astype("category").cat.codes
    df["IsHoliday"] = df["IsHoliday"].astype(int)
    df.fillna(0, inplace=True)
    return df

train = preprocess(train)
test = preprocess(test, is_train=False)

# Prepare Features and Target


In [ ]:
features_cols = [
    "Store", "Dept", "IsHoliday", "Temperature", "Fuel_Price",
    "CPI", "Unemployment", "Size", "Type", "Year", "Month", "Week",
    "MarkDown1", "MarkDown2", "MarkDown3", "MarkDown4", "MarkDown5"
]

train = train.dropna(subset=["Weekly_Sales"])  # Ensure target has no NaNs
X_train_np = train[features_cols].values.astype(np.float32)
y_train_np = np.log1p(train["Weekly_Sales"].values.astype(np.float32))  # log transform

X_test_np = test[features_cols].values.astype(np.float32)

# Convert to PyTorch Tensors


In [ ]:
X_train_tensor = torch.tensor(X_train_np)
y_train_tensor = torch.tensor(y_train_np).view(-1, 1)
X_test_tensor = torch.tensor(X_test_np)

# Create Dataset and DataLoader


In [ ]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)

ModuleNotFoundError: No module named 'torch'

# Simple Linear Regression Model


In [ ]:
model = nn.Linear(X_train_tensor.shape[1], 1)

# Loss and Optimizer


In [ ]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Train loop


In [ ]:
epochs = 20
for epoch in range(epochs):
    model.train()
    epoch_loss = 0.0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs} | Loss: {epoch_loss:.4f}")

# Make predictions


In [ ]:
model.eval()
with torch.no_grad():
    test_preds_log = model(X_test_tensor).squeeze().numpy()
    test_preds = np.expm1(test_preds_log)  # Reverse log1p
    
#Create Id column in the format store_dept_date
submission = test.copy()
submission['Id'] = submission['Store'].astype(str) + "" + \
                   submission['Dept'].astype(str) + "" + \
                   submission['Date'].dt.strftime("%Y-%m-%d")

#Add predictions
submission['Weekly_Sales'] = predictions

#Select required columns and save
submission[['Id', 'Weekly_Sales']].to_csv("Submission.csv", index=False)